## Snowflake Feedback Functions
This example notebook computes feedback functions in deferred mode inside snowflake warehouse. 
Trulens library imported in the LLM app just takes care of logging to snowflake.

In [ ]:
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.schema.app import RecordIngestMode
from trulens.core.session import TruSession

connection_params = {
    "account": "xxx",
    "user": "xxx",
    "password": "xxxx",
    "database": "xxx",
    "schema": "xxx",
    "warehouse": "xxx",
    "role": "ENGINEER",
    "init_server_side": True,  # Set to True to enable server side feedback functions
}

connector = SnowflakeConnector(**connection_params)
session = TruSession(connector=connector)

In [ ]:
from snowflake.snowpark import Session

snowpark_session = Session.builder.configs(connection_params).create()

In [ ]:
uw_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

wsu_info = """
Washington State University, commonly known as WSU, founded in 1890, is a public research university in Pullman, Washingt
on.
With multiple campuses across the state, it is the state's second largest institution of higher education.
WSU is known for its programs in veterinary medicine, agriculture, engineering, architecture, and pharmacy.
"""

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")
document_embeddings = model.encode([uw_info])

In [ ]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Univerisities")
vector_store.add("uni_info", documents=uw_info, embeddings=document_embeddings)

In [ ]:
import json

from trulens.core.instruments import instrument


class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
            query_embeddings=model.encode([query], prompt_name="query"),
            n_results=2,
        )
        return results["documents"]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """

        def escape_string_for_sql(input_string):
            escaped_string = input_string.replace("\\", "\\\\")
            escaped_string = escaped_string.replace("'", "''")
            return escaped_string

        prompt = escape_string_for_sql(f"""
         We have provided context information below. 
            {context_str}
            Given this information, please answer the question: {query}
        """)

        res = snowpark_session.sql(f"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'snowflake-arctic',
            [
            {{'role': 'user', 'content': '{prompt}'}}
            ], {{
                'temperature': 0
            }}
            )""").collect()

        if len(res) == 0:
            return "No response from cortex function"
        completion = json.loads(res[0][0])["choices"][0]["messages"]
        print("full response from cortex function:")
        print(res)
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion


rag = RAG_from_scratch()

In [ ]:
import numpy as np
from trulens.core import Select
from trulens.core.feedback.feedback import SnowflakeFeedback
from trulens.providers.cortex import Cortex

provider = Cortex(
    connection_params,
    model_engine="snowflake-arctic",
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    SnowflakeFeedback(
        provider.relevance_with_cot_reasons, name="Answer Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    SnowflakeFeedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    app_version="v1",
    feedbacks=[
        f_answer_relevance,
        f_context_relevance,
    ],
    record_ingest_mode=RecordIngestMode.BUFFERED,
)

In [ ]:
with tru_rag as recording:
    for i in range(10):
        resp = rag.query(
            f"Question {i} When is University of Washington founded ?"
        )
        print(resp)